### 0) Data Importing and Helper Function Definition:

In [1]:
import numpy as np

DATA_folder  = '../../Data/'
data = np.load(DATA_folder+'train_imgs.npy')
lbls = np.load(DATA_folder+'train_lbls.npy')
test_data = np.load(DATA_folder+'test_imgs.npy')
test_lbls = np.load(DATA_folder+'test_lbls.npy')

In [2]:
# http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html

### 1) Definition of RBF Transformer Object:

RBF Transformer is a scikit-learn compatible transformer object that implements:

    - fit method       - clusters the digit-separated data, and computes cluster centers and inv. sq. deviations
    - transform method - based on obtained centers and deviations it computes the fi values (RBF layer outputs) as 
                         simple Gaussian functions -> Fi_k(x) = exp{ -sqrt( sum[ (x_i-c_k_i)/(dev_i^2) ] ) } 
                         
## Normalizing each Fi row with row sum -> STABILIZED the Regression a lot!!!

## For PCA fit_transform() method needs to be separated to fit() and then transform()!!!

#### Definition of Clustering Function:

In [3]:
### Clustering:

# from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.cluster import KMeans, SpectralClustering
import random

# clust = AgglomerativeClustering(n_clusters=500, linkage='complete')
# cluster_labels = clust.fit_predict(dig_data)

# Function for data clustering, and computation of cluster center vectors and inv. sq. deviation vectors
def form_clusters(data, n_kmeans, n_agglo, n_random, dig):
    
    n_km = min(n_kmeans,data.shape[0])
    n_ag = min(n_agglo ,data.shape[0])
    n_rn = min(n_random,data.shape[0])
    
    lbls_set = []
    
    if n_km>0:
        kmeans = KMeans(n_clusters=n_km, random_state=0, init='k-means++', algorithm='elkan')
        lbls_kmeans = kmeans.fit_predict(data)
        lbls_set.append(lbls_kmeans)
    
    if n_ag>0:
        #agglo  = SpectralClustering(n_clusters=n_ag, affinity='nearest_neighbors',eigen_solver='arpack',random_state=456,assign_labels='discretize')
        agglo  = SpectralClustering(n_clusters=n_ag, affinity='sigmoid',eigen_solver='arpack',random_state=456,assign_labels='discretize')
        lbls_agglo  = agglo.fit_predict(data)
        lbls_set.append(lbls_agglo)
    
    centers     = []
    
    # Find cluster centers and covar matrix:
    for lbls in lbls_set:
        for k in range(max(lbls)+1):
            cluster = data[lbls==k,:]
            centers.append(cluster.mean(axis=0))
            
        del cluster
        
    # Add random points as centers:
    if n_rn>0:
        # seed a new random generator (to get repeatable results for hyperparam tuning)
        prng = np.random.RandomState()
        prng.seed(dig*n_rn)
        smpls = prng.choice(data.shape[0],n_rn,replace=False)
        for s in smpls:
            centers.append(data[s,:])
        
    return centers;


In [4]:
a = np.ones((4,5))
a[1,1]=5
a.max()

5.0

#### Definition of RBF (Fi) Function:

In [5]:
import gc

# chunks of 100, (10k smpls, 500 centers) ->  110.8s
# chunks of 200, (10k smpls, 500 centers) ->  102.3s
# chunks of 250, (10k smpls, 500 centers) ->  115.6s
# chunks of 500, (10k smpls, 500 centers) ->  145.1s

# Function to compute whole Fi output for given dataset (for all RB centers)
def fi_transform(data, all_centers):
    # data        - given dataset matrix for which to compute fi values
    # center      - list of all center vectors on which to compute fi vals

    new_data = np.empty((data.shape[0],len(all_centers)))
    st = 0
    ch = 200
    ns = data.shape[0]
    # Process data in chunks of 200 smpls (optimal speed)
    while st<ns:
        en = min(st+ch,ns)
        for k in range(len(all_centers)):
            # # SAME DEVIATION for ALL DIMENSIONS (better much better than cluster separate):
            new_data[st:en,k] = (np.square(data[st:en] - np.repeat(all_centers[k][np.newaxis,:],en-st,axis=0))).mean(axis=1)
        new_data[st:en,:] = np.exp(-np.sqrt(new_data[st:en,:]))
        st = en
        gc.collect()
    #print('\n',new_data.max())
    #new_data = np.exp(-np.sqrt(new_data))
    return new_data

### Definition od Kernel PCA Layer

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import StandardScaler
import time, gc

# SKLEARN Compatible Transformer - supports fit method (finding kPCA) and transform method (applying kPCA)
class myKernelPCA(BaseEstimator, TransformerMixin):
    
    # Transformer initialization (default to 250 dimensions)
    def __init__(self, n_kPCA=250, kernel = 'poly', degree=4, whiten=True, debug=False):
        self.n_kPCA    = n_kPCA # number of kernel PCA dimensions to be retained
        self.kernel    = kernel # kernel type (rbf, poly, cosine)
        self.degree    = degree # poly kernel degree
        #self.whiten    = whiten # whiten data at output layer (zero mean, un var)
        self.whiten    = whiten and (kernel!='laplacian') and (kernel!='rbf')
        self.debug     = debug  # debug flag
        self.max_smpls = 250    # max number of samples per digit
    
    # Clusters each digit and finds cluster centers:
    def fit(self, X, y):
        
        self.kPCA = KernelPCA(n_components=self.n_kPCA,kernel=self.kernel,degree=self.degree,copy_X=False,remove_zero_eig=True,random_state=1389)
        
        if self.debug:
            print('Applying Kernel PCA !!!')
            t = time.time()
        
        # If we have less than max number of smpls - fit regular kPCA
        if X.shape[0]<(self.max_smpls*10):
            self.kPCA.fit(X)
            # If required - whiten the data (for clustering)
            if self.whiten:
                self.scaler = StandardScaler().fit(self.kPCA.transform(X))
        
        # Else randomly pick max number of smpls for each digit and fit
        else:
            # Fix the random seed (for repeatable results)
            prng = np.random.RandomState()
            prng.seed(654)
            # Check how many smps are available per digit
            l = []
            for dig in range(10):
                l.append(min(self.max_smpls,sum(y==dig)))
            # Preallocate memory
            new_data = np.empty((sum(l),X.shape[1]))
            # Fill with randomly drawn smpls
            st = 0
            for dig in range(10):
                en = st+l[dig]
                data  = X[y==dig]
                smpls = prng.choice(data.shape[0],l[dig],replace=False)
                new_data[st:en,] = data[smpls,]
                st = en
            # Fit KernelPCA to new_data
            self.kPCA.fit(new_data)
            # If required - whiten the data (for clustering)
            if self.whiten:
                self.scaler = StandardScaler().fit(self.kPCA.transform(new_data))
            # Release old vars, and collect garbage
            del data, smpls, new_data
            gc.collect()
            
        if self.debug:
            print('Kernel PCA fitted in: ',(time.time()-t))
        
        return self
    
    # Clculate Kernel Components based on obtained model
    def transform(self, X, y=None):
        
        if self.debug:
            print('Applying Kernel PCA transformation!!! (data', X.shape,')')
            t = time.time()
        # Preallocate memory:
        new_data = np.empty((X.shape[0],self.kPCA.lambdas_.shape[0]))
        st = 0
        ch = 200
        ns = X.shape[0]
        # Transform data in chunks of 200 smpls (memory hungry transformation)
        while st<ns:
            en = min(st+ch,ns)
            new_data[st:en,:] = self.kPCA.transform(X[st:en,:])
            st = en
        
        if self.whiten:
            new_data = self.scaler.transform(new_data)
                
        if self.debug:
            print('Kernel PCA transformed in: ',time.time()-t)
        
        return new_data
    
    def fit_transform(self, X, y):
        self.fit(X,y)
        return self.transform(X)

#### Definition of RBF Transformer (sklearn compatible object):

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
import time, gc

# SKLEARN Compatible Transformer - supports fit method (custering data) and transform method (calculating fi values)
class myRBFtransformer(BaseEstimator, TransformerMixin):
    
    # Transformer initialization (default to 50 kMeans clusters)
    def __init__(self, n_centers=250, rn_ratio=0.1, cl_ratio=0.5, debug=False):
        self.n_centers = n_centers              # number of data centers to be formed per digit
        self.rn_ratio  = min(1,max(0,rn_ratio)) # random  ratio (between Random Selection and Clustering)
        self.cl_ratio  = min(1,max(0,cl_ratio)) # cluster ratio (between kMeans and Agglomerative)
        self.debug     = debug                  # debug flag
        if self.debug:
            print(self.n_centers,self.cl_ratio)
    
    # Clusters each digit and finds cluster centers:
    def fit(self, X, y):
        
        self.centers     = [] # list of cluster center vectors
        self.num_centers = [] # list of number of centers per digit
        
        # Calc. num. of clusters per digit based on assigned ratio (ratio*(num of smpls / 10))
        n_random   = round(self.rn_ratio*self.n_centers)
        n_clusters = self.n_centers - n_random
        n_kmeans   = round(self.cl_ratio*n_clusters)
        n_agglo    = n_clusters - n_kmeans
        
        if self.debug:
            print('Clustering data ',(n_kmeans,n_agglo))
            t = time.time()
        
        # Cluster the data over each digit
        for dig in range(10):
            # print('Clustering digit: ',dig)
            data = X[y==dig,:]
            centers = form_clusters(data, n_kmeans, n_agglo, n_random, dig)
            self.centers.extend(centers)
            self.num_centers.append(len(centers))
            
        if self.debug:
            print('Clustering time: ',(time.time()-t))
        
        del data, centers
        gc.collect()
        
        return self
    
    # Computes fi values (with Gaussian function) based on obtained data centers
    def transform(self, X, y=None):
        
        # Compute all Fi values:
        if self.debug:
            print('Calculating all Fi outputs !!! (data', X.shape,')')
            t = time.time()
        all_fis = fi_transform(X, self.centers)
        
        if self.debug:
            print('Fi calculation time: ',time.time()-t)
        
        return all_fis
    
    def fit_transform(self, X, y):
        self.fit(X,y)
        return self.transform(X)

In [8]:
# # Test:
# rbf = myRBFtransformer(n_kmeans = 10)
# a   = rbf.fit_transform(X=data[:500,:],y=lbls[:500])
# a.max(axis=1)

### 3) Pipelining the Model:

In [9]:
import types
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model  import LogisticRegression

n_kPCA    = 300
degree    = 3
kernel    = 'laplacian'
n_centers = 50
rn_ratio  = 0.3
cl_ratio  = 0.3
cl_ratio  = 1.0
C         = 1e3
tol       = 1e-6
debug     = False

scaler = StandardScaler()
pca    = myKernelPCA(kernel=kernel,degree=degree,n_kPCA=n_kPCA,debug=debug)
rbf    = myRBFtransformer(n_centers=n_centers,rn_ratio=rn_ratio,cl_ratio=cl_ratio, debug=debug)
logreg = LogisticRegression(tol=tol,C=C,random_state=12,solver='liblinear')


In [10]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

pipe = Pipeline(steps=[('scal', scaler), ('pca', pca), ('rbf',rbf), ('logreg',logreg)])

In [11]:
# from sklearn.model_selection import GridSearchCV
from dask_searchcv import GridSearchCV

Ks  = [1000]
n_jobs = 1
RNs = [0.0,0.25,0.5,0.75,1.0] # Random Ratio
CLs = [1.0] # Cluster Ratio
Cs  = [1e18]
# Cs  = [1e3]
tols= [1e-6]
NKs = [25]

kernels = ['laplacian']

param_grid = [
    {
        #'pca__kernel':    kernels,
        'pca__n_kPCA':    NKs,
        'rbf__n_centers': Ks,
        'rbf__rn_ratio':  RNs,
        'rbf__cl_ratio':  CLs,
        'logreg__C':      Cs,
        #'logreg__tol':    tols
    }
]

#         'rbf__n_centers': Ks,
#         'rbf__rn_ratio':  RNs,
#         'rbf__cl_ratio':  CLs,
#         'logreg__C':      Cs

In [12]:
from sklearn.model_selection import PredefinedSplit
import gc

test_fold = [-1]*data.shape[0]+[0]*test_data.shape[0]
ps = PredefinedSplit(test_fold)

tot_data = np.vstack((data,test_data))
tot_lbls = np.hstack((lbls,test_lbls))

del data,test_data,lbls,test_lbls
gc.collect()

17

In [13]:
grid = GridSearchCV(pipe, cv=ps, n_jobs=n_jobs, param_grid=param_grid, refit=False, return_train_score=False)

In [14]:
from dask.diagnostics import ProgressBar

In [ ]:
### Different Kernels (logreg: tol = 1e-3, C=1e3; 60k trainset):
# ------------------
# FULL kMEANS:
# 
# laplacian - 94.46%
# rbf       - 93.24%
# poly_3    - 92.95%
# sigmoid   - 91.72%
# poly_6    - 90.82%
# ------------------


In [ ]:
with ProgressBar():
    #grid.fit(data, lbls)
    grid.fit(tot_data, tot_lbls)

print(max(grid.cv_results_['mean_test_score'])*100)

[#                                       ] | 2% Completed |  1.8s

In [ ]:
grid.param_grid

In [ ]:
print('### Parameter Grid:\n')
for k in range(len(grid.cv_results_['params'])):
    print(grid.cv_results_['params'][k])
    sc  = np.round(grid.cv_results_['mean_test_score'][k]*100,2)
    ftm = np.round(grid.cv_results_['mean_fit_time'][k])
    stm = np.round(grid.cv_results_['mean_score_time'][k])
    #print('Score:',sc,'%; Fit time:',ftm,'s; Score Time:',stm,'s;\n')
    tsc = np.round(grid.cv_results_['mean_train_score'][k]*100,2)
    print('Test Score:',sc,'%; Train Score:',tsc,'%; Fit time:',ftm,'s; Score Time:',stm,'s;\n')

# print('\n### Mean Fit Times:')
# print(np.round(grid.cv_results_['mean_fit_time']))
# print('\n### Mean Score Times')
# print(np.round(grid.cv_results_['mean_score_time']))
# # print('\n### Mean Train Score')
# # print(np.round(grid.cv_results_['mean_train_score']*100,2))
# print('\n### Mean Test Score')
# print(np.round(grid.cv_results_['mean_test_score']*100,2))

In [ ]:
# ### kPCA = 250; TOLERANCE = 1e-6 ### #

# ### Grid Search Results (Different Num of Centers):

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 50, 'rbf__rn_ratio': 0.0}
# Score: 93.83 %; Fit time: 1217.0 s; Score Time: 74.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 50, 'rbf__rn_ratio': 0.25}
# Score: 93.89 %; Fit time: 1196.0 s; Score Time: 72.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 50, 'rbf__rn_ratio': 0.5}
# Score: 93.86 %; Fit time: 4081.0 s; Score Time: 518.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 50, 'rbf__rn_ratio': 0.75}
# Score: 93.92 %; Fit time: 1563.0 s; Score Time: 159.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 50, 'rbf__rn_ratio': 1.0}
# Score: 93.77 %; Fit time: 1899.0 s; Score Time: 339.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 75, 'rbf__rn_ratio': 0.0}
# Score: 94.27 %; Fit time: 3825.0 s; Score Time: 256.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 75, 'rbf__rn_ratio': 0.25}
# Score: 94.31 %; Fit time: 2324.0 s; Score Time: 189.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 75, 'rbf__rn_ratio': 0.5}
# Score: 94.33 %; Fit time: 2219.0 s; Score Time: 205.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 75, 'rbf__rn_ratio': 0.75}
# Score: 94.19 %; Fit time: 2143.0 s; Score Time: 135.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 75, 'rbf__rn_ratio': 1.0}
# Score: 94.28 %; Fit time: 1312.0 s; Score Time: 62.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.0}
# Score: 94.49 %; Fit time: 1980.0 s; Score Time: 100.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.25}
# Score: 94.45 %; Fit time: 1934.0 s; Score Time: 98.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 94.46 %; Fit time: 1842.0 s; Score Time: 94.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.75}
# Score: 94.55 %; Fit time: 1797.0 s; Score Time: 91.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 1.0}
# Score: 94.51 %; Fit time: 1577.0 s; Score Time: 96.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 125, 'rbf__rn_ratio': 0.0}
# Score: 94.66 %; Fit time: 2510.0 s; Score Time: 1699.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 125, 'rbf__rn_ratio': 0.25}
# Score: 94.76 %; Fit time: 2537.0 s; Score Time: 1523.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 125, 'rbf__rn_ratio': 0.5}
# Score: 94.69 %; Fit time: 2514.0 s; Score Time: 1524.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 125, 'rbf__rn_ratio': 0.75}
# Score: 94.73 %; Fit time: 2195.0 s; Score Time: 1418.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 125, 'rbf__rn_ratio': 1.0}
# Score: 94.79 %; Fit time: 9405.0 s; Score Time: 1550.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 150, 'rbf__rn_ratio': 0.0}
# Score: 94.89 %; Fit time: 9184.0 s; Score Time: 1656.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 150, 'rbf__rn_ratio': 0.25}
# Score: 94.84 %; Fit time: 9107.0 s; Score Time: 1656.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 150, 'rbf__rn_ratio': 0.5}
# Score: 94.84 %; Fit time: 4831.0 s; Score Time: 1605.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 150, 'rbf__rn_ratio': 0.75}
# Score: 94.82 %; Fit time: 11003.0 s; Score Time: 685.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 150, 'rbf__rn_ratio': 1.0}
# Score: 94.81 %; Fit time: 10159.0 s; Score Time: 795.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 175, 'rbf__rn_ratio': 0.0}
# Score: 94.99 %; Fit time: 13333.0 s; Score Time: 921.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 175, 'rbf__rn_ratio': 0.25}
# Score: 94.96 %; Fit time: 12847.0 s; Score Time: 1175.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 175, 'rbf__rn_ratio': 0.5}
# Score: 94.9 %; Fit time: 7480.0 s; Score Time: 1871.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 175, 'rbf__rn_ratio': 0.75}
# Score: 94.85 %; Fit time: 7169.0 s; Score Time: 2021.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 175, 'rbf__rn_ratio': 1.0}
# Score: 94.98 %; Fit time: 5389.0 s; Score Time: 1080.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 200, 'rbf__rn_ratio': 0.0}
# Score: 95.11 %; Fit time: 4527.0 s; Score Time: 4260.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 200, 'rbf__rn_ratio': 0.25}
# Score: 95.05 %; Fit time: 4527.0 s; Score Time: 4255.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 200, 'rbf__rn_ratio': 0.5}
# Score: 95.08 %; Fit time: 4007.0 s; Score Time: 4186.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 200, 'rbf__rn_ratio': 0.75}
# Score: 94.98 %; Fit time: 4486.0 s; Score Time: 3525.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 200, 'rbf__rn_ratio': 1.0}
# Score: 95.0 %; Fit time: 4121.0 s; Score Time: 3528.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 225, 'rbf__rn_ratio': 0.0}
# Score: 95.15 %; Fit time: 4697.0 s; Score Time: 3251.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 225, 'rbf__rn_ratio': 0.25}
# Score: 95.18 %; Fit time: 4652.0 s; Score Time: 3251.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 225, 'rbf__rn_ratio': 0.5}
# Score: 95.16 %; Fit time: 3835.0 s; Score Time: 1713.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 225, 'rbf__rn_ratio': 0.75}
# Score: 95.06 %; Fit time: 3833.0 s; Score Time: 1715.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 225, 'rbf__rn_ratio': 1.0}
# Score: 95.12 %; Fit time: 3534.0 s; Score Time: 1712.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.0}
# Score: 95.25 %; Fit time: 5207.0 s; Score Time: 3845.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.25}
# Score: 95.25 %; Fit time: 5207.0 s; Score Time: 3845.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Score: 95.21 %; Fit time: 5134.0 s; Score Time: 3694.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.75}
# Score: 95.36 %; Fit time: 5129.0 s; Score Time: 3695.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 250, 'rbf__rn_ratio': 1.0}
# Score: 95.11 %; Fit time: 4864.0 s; Score Time: 3694.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 275, 'rbf__rn_ratio': 0.0}
# Score: 95.39 %; Fit time: 8260.0 s; Score Time: 3006.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 275, 'rbf__rn_ratio': 0.25}
# Score: 95.36 %; Fit time: 6257.0 s; Score Time: 4447.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 275, 'rbf__rn_ratio': 0.5}
# Score: 95.32 %; Fit time: 6159.0 s; Score Time: 4530.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 275, 'rbf__rn_ratio': 0.75}
# Score: 95.37 %; Fit time: 6159.0 s; Score Time: 4538.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 275, 'rbf__rn_ratio': 1.0}
# Score: 95.32 %; Fit time: 8077.0 s; Score Time: 170.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.0}
# Score: 95.81 %; Fit time: 5046.0 s; Score Time: 92.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.25}
# Score: 95.78 %; Fit time: 6801.0 s; Score Time: 157.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.5}
# Score: 95.78 %; Fit time: 4823.0 s; Score Time: 92.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.75}
# Score: 95.74 %; Fit time: 6471.0 s; Score Time: 475.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 500, 'rbf__rn_ratio': 1.0}
# Score: 95.75 %; Fit time: 4672.0 s; Score Time: 91.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 1000, 'rbf__rn_ratio': 0.0}
# Score: 96.47 %; Fit time: 12450.0 s; Score Time: 643.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 1000, 'rbf__rn_ratio': 1.0}
# Score: 96.29 %; Fit time: 13492.0 s; Score Time: 698.0 s;



In [ ]:
# ### kPCA = 500; TOLERANCE = 1e-6 ### #

# ### Grid Search Results (Different Num of kPCA dimensions):

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 5, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 75.44 %; Fit time: 4068.0 s; Score Time: 32.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 10, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 91.89 %; Fit time: 3636.0 s; Score Time: 42.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 15, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 94.88 %; Fit time: 3350.0 s; Score Time: 55.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 20, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 95.66 %; Fit time: 3131.0 s; Score Time: 74.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.0}
# Score: 95.8 %; Fit time: 2958.0 s; Score Time: 60.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.25}
# Score: 95.71 %; Fit time: 3076.0 s; Score Time: 59.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 95.89 %; Fit time: 3071.0 s; Score Time: 78.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.75}
# Score: 95.86 %; Fit time: 3066.0 s; Score Time: 82.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 1.0}
# Score: 95.81 %; Fit time: 2975.0 s; Score Time: 65.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 50, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.0}
# Score: 95.73 %; Fit time: 2380.0 s; Score Time: 68.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 50, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.25}
# Score: 95.73 %; Fit time: 2266.0 s; Score Time: 63.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 50, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 95.82 %; Fit time: 2312.0 s; Score Time: 63.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 50, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.75}
# Score: 95.63 %; Fit time: 2327.0 s; Score Time: 69.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 50, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 1.0}
# Score: 95.67 %; Fit time: 2207.0 s; Score Time: 97.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 75, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.0}
# Score: 95.31 %; Fit time: 2409.0 s; Score Time: 68.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 75, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.25}
# Score: 95.29 %; Fit time: 2371.0 s; Score Time: 87.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 75, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 95.22 %; Fit time: 2268.0 s; Score Time: 90.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 75, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.75}
# Score: 95.27 %; Fit time: 2305.0 s; Score Time: 64.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 75, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 1.0}
# Score: 95.36 %; Fit time: 2244.0 s; Score Time: 84.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 100, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.0}
# Score: 95.02 %; Fit time: 2360.0 s; Score Time: 70.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 100, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.25}
# Score: 95.06 %; Fit time: 2313.0 s; Score Time: 80.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 100, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 95.03 %; Fit time: 2239.0 s; Score Time: 82.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 100, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.75}
# Score: 95.05 %; Fit time: 2227.0 s; Score Time: 69.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 100, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 1.0}
# Score: 95.09 %; Fit time: 2100.0 s; Score Time: 107.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 125, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.0}
# Score: 94.92 %; Fit time: 2198.0 s; Score Time: 73.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 125, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.25}
# Score: 94.9 %; Fit time: 2316.0 s; Score Time: 76.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 125, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 94.87 %; Fit time: 2179.0 s; Score Time: 76.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 125, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.75}
# Score: 94.97 %; Fit time: 2142.0 s; Score Time: 74.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 125, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 1.0}
# Score: 94.92 %; Fit time: 1857.0 s; Score Time: 74.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 150, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.0}
# Score: 94.75 %; Fit time: 2034.0 s; Score Time: 87.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 150, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.25}
# Score: 94.7 %; Fit time: 2037.0 s; Score Time: 89.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 150, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 94.72 %; Fit time: 1550.0 s; Score Time: 46.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 150, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.75}
# Score: 94.83 %; Fit time: 1543.0 s; Score Time: 48.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 150, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 1.0}
# Score: 94.84 %; Fit time: 1423.0 s; Score Time: 55.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 175, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.0}
# Score: 94.66 %; Fit time: 2071.0 s; Score Time: 93.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 175, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.25}
# Score: 94.6 %; Fit time: 2020.0 s; Score Time: 93.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 175, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 94.68 %; Fit time: 2004.0 s; Score Time: 135.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 175, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.75}
# Score: 94.74 %; Fit time: 1981.0 s; Score Time: 94.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 175, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 1.0}
# Score: 94.74 %; Fit time: 1824.0 s; Score Time: 122.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 200, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.0}
# Score: 94.6 %; Fit time: 2730.0 s; Score Time: 182.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 200, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.25}
# Score: 94.6 %; Fit time: 2673.0 s; Score Time: 150.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 200, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 94.58 %; Fit time: 1791.0 s; Score Time: 53.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 200, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.75}
# Score: 94.56 %; Fit time: 1736.0 s; Score Time: 81.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 200, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 1.0}
# Score: 94.67 %; Fit time: 1683.0 s; Score Time: 80.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 250, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.0}
# Score: 94.49 %; Fit time: 1980.0 s; Score Time: 100.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 250, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.25}
# Score: 94.45 %; Fit time: 1934.0 s; Score Time: 98.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 250, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 94.46 %; Fit time: 1842.0 s; Score Time: 94.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 250, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.75}
# Score: 94.55 %; Fit time: 1797.0 s; Score Time: 91.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 250, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 1.0}
# Score: 94.51 %; Fit time: 1577.0 s; Score Time: 96.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 300, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.0}
# Score: 94.43 %; Fit time: 2736.0 s; Score Time: 118.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 300, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.25}
# Score: 94.39 %; Fit time: 2249.0 s; Score Time: 178.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 300, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 94.32 %; Fit time: 2136.0 s; Score Time: 121.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 300, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.75}
# Score: 94.43 %; Fit time: 2157.0 s; Score Time: 152.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 300, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 1.0}
# Score: 94.41 %; Fit time: 1936.0 s; Score Time: 120.0 s;

# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 400, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.0}
# Score: 94.24 %; Fit time: 2710.0 s; Score Time: 160.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 400, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.25}
# Score: 94.16 %; Fit time: 2432.0 s; Score Time: 182.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 400, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 94.21 %; Fit time: 2372.0 s; Score Time: 151.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 400, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.75}
# Score: 94.25 %; Fit time: 2319.0 s; Score Time: 180.0 s;
# {'logreg__C': 1000.0, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 400, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 1.0}
# Score: 94.25 %; Fit time: 2105.0 s; Score Time: 224.0 s;




In [ ]:
# ### kPCA = 25; RN = 0.5; N_cent = 100 ### #

# ### TOLERANCE = 1e-6 ### #
# {'logreg__C': 1e+03, 'logreg__tol': 1e-06, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 95.89 %; Fit time: 2624.0 s; Score Time: 62.0 s;
# {'logreg__C': 1e+06, 'logreg__tol': 1e-06, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.58 %; Fit time: 10547.0 s; Score Time: 66.0 s;
# {'logreg__C': 1e+09, 'logreg__tol': 1e-06, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.58 %; Fit time: 10491.0 s; Score Time: 64.0 s;
# {'logreg__C': 1e+12, 'logreg__tol': 1e-06, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.58 %; Fit time: 10207.0 s; Score Time: 63.0 s;
# {'logreg__C': 1e+15, 'logreg__tol': 1e-06, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.53 %; Fit time: 9752.0 s; Score Time: 51.0 s;
# {'logreg__C': 1e+18, 'logreg__tol': 1e-06, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.63 %; Fit time: 6637.0 s; Score Time: 39.0 s;
# {'logreg__C': 1e+21, 'logreg__tol': 1e-06, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.55 %; Fit time: 6408.0 s; Score Time: 33.0 s;

# ### TOLERANCE = 1e-9 ### #
# {'logreg__C': 1e+03, 'logreg__tol': 1e-09, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 95.89 %; Fit time: 3490.0 s; Score Time: 60.0 s;
# {'logreg__C': 1e+06, 'logreg__tol': 1e-09, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.58 %; Fit time: 10733.0 s; Score Time: 70.0 s;
# {'logreg__C': 1e+09, 'logreg__tol': 1e-09, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.58 %; Fit time: 10577.0 s; Score Time: 68.0 s;
# {'logreg__C': 1e+12, 'logreg__tol': 1e-09, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.58 %; Fit time: 10343.0 s; Score Time: 77.0 s;
# {'logreg__C': 1e+15, 'logreg__tol': 1e-09, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.53 %; Fit time: 9175.0 s; Score Time: 38.0 s;
# {'logreg__C': 1e+18, 'logreg__tol': 1e-09, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.63 %; Fit time: 10059.0 s; Score Time: 75.0 s;
# {'logreg__C': 1e+21, 'logreg__tol': 1e-09, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.55 %; Fit time: 10127.0 s; Score Time: 61.0 s;

# ### TOLERANCE = 1e-12 ### #
# {'logreg__C': 1e+03, 'logreg__tol': 1e-12, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 95.89 %; Fit time: 3829.0 s; Score Time: 60.0 s;
# {'logreg__C': 1e+06, 'logreg__tol': 1e-12, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.58 %; Fit time: 10714.0 s; Score Time: 66.0 s;
# {'logreg__C': 1e+09, 'logreg__tol': 1e-12, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.58 %; Fit time: 10608.0 s; Score Time: 68.0 s;
# {'logreg__C': 1e+12, 'logreg__tol': 1e-12, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.58 %; Fit time: 10315.0 s; Score Time: 80.0 s;
# {'logreg__C': 1e+15, 'logreg__tol': 1e-12, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.53 %; Fit time: 8963.0 s; Score Time: 33.0 s;
# {'logreg__C': 1e+18, 'logreg__tol': 1e-12, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.63 %; Fit time: 10058.0 s; Score Time: 75.0 s;
# {'logreg__C': 1e+21, 'logreg__tol': 1e-12, 'pca__kernel': 'laplacian', 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Score: 97.55 %; Fit time: 10144.0 s; Score Time: 61.0 s;

In [ ]:
# ### kPCA = 25; RN = 0.5; N_cent = 100 ### #

# ### TOLERANCE = 1e-6 ### #
# {'logreg__C': 1e03, 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Test Score: 95.89 %; Train Score: 96.11 %; Fit time: 2577.0 s; Score Time: 69.0 s;
# {'logreg__C': 1e06, 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Test Score: 97.58 %; Train Score: 98.56 %; Fit time: 10610.0 s; Score Time: 61.0 s;
# {'logreg__C': 1e09, 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Test Score: 97.58 %; Train Score: 98.52 %; Fit time: 10617.0 s; Score Time: 65.0 s;
# {'logreg__C': 1e12, 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Test Score: 97.58 %; Train Score: 98.5 %; Fit time: 10414.0 s; Score Time: 64.0 s;
# {'logreg__C': 1e15, 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Test Score: 97.53 %; Train Score: 98.5 %; Fit time: 10343.0 s; Score Time: 64.0 s;
# {'logreg__C': 1e+16, 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Test Score: 97.57 %; Train Score: 98.52 %; Fit time: 10558.0 s; Score Time: 66.0 s;
# {'logreg__C': 1e+17, 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Test Score: 97.65 %; Train Score: 98.57 %; Fit time: 10811.0 s; Score Time: 61.0 s;
# {'logreg__C': 1e+18, 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Test Score: 97.63 %; Train Score: 98.54 %; Fit time: 9089.0 s; Score Time: 50.0 s;
# {'logreg__C': 1e+19, 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Test Score: 97.56 %; Train Score: 98.54 %; Fit time: 9291.0 s; Score Time: 59.0 s;
# {'logreg__C': 1e+20, 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Test Score: 97.58 %; Train Score: 98.55 %; Fit time: 9236.0 s; Score Time: 38.0 s;
# {'logreg__C': 1e+21, 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Test Score: 97.55 %; Train Score: 98.5 %; Fit time: 10238.0 s; Score Time: 59.0 s;
# {'logreg__C': 1e+24, 'pca__n_kPCA': 25, 'rbf__cl_ratio': 1.0, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Test Score: 97.6 %; Train Score: 98.52 %; Fit time: 10398.0 s; Score Time: 62.0 s;